# Organização de Vídeos ICT 2019-2020

## Inicialização

In [1]:
# Incluindo diretório scripts no caminho
from glob import glob
import sys
import os
sys.path.insert(0, './DATA/')

In [2]:
# show result from all calculations of the cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
# pacotes básicos
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('max_colwidth',250)
import unidecode
import re

## Carregando dados

In [58]:
colunas = ['ID', 'Título do Projeto', 'Docente', 'Centro', 'Unidade', 
           'Área Conhecimento', 'Aluno', 'CPF Aluno', 'Cota Bolsa','TIPO']

In [59]:
servidores = pd.read_excel('./DATA/UFCG_Servidores_Ativos.xls', converters={'CPF':str})
servidores = servidores[['CPF','Nome']]


formularios = pd.read_csv('./DATA/Formulário_ICT_VIDEOS.csv', converters={'CPF_ALUNO':str, 'CPF_ORIENTADOR':str})
formularios = formularios.dropna(axis=1, how='any')
formularios['CPF_ORIENTADOR'] = formularios['CPF_ORIENTADOR'].apply(
    lambda row: re.sub(r"\D", "", row))
formularios['CPF_ALUNO'] = formularios['CPF_ALUNO'].apply(
    lambda row: re.sub(r"\D", "", row))
formularios['TIMESTAMP'] = pd.to_datetime(formularios['TIMESTAMP'])

PIBIC = pd.read_excel('./DATA/PIBIC-2019.xls', converters={'CPF Aluno':str})
PIVIC = pd.read_excel('./DATA/PIVIC-2019.xls', converters={'CPF Aluno':str})
PIVIC_1 = pd.read_excel('./DATA/PIVIC-2019-1.xls', converters={'CPF Aluno':str})
PIBITI = pd.read_excel('./DATA/PIBITI-2019.xls', converters={'CPF Aluno':str})
PIVITI = pd.read_excel('./DATA/PIVITI-2019.xls', converters={'CPF Aluno':str})

PIBIC['TIPO'] = 'PIBIC' 
PIVIC['TIPO'] = 'PIVIC' 
PIVIC_1['TIPO'] = 'PIVIC' 
PIBITI['TIPO'] = 'PIBITI' 
PIVITI['TIPO'] = 'PIVITI' 

projetos = pd.concat([PIBIC,PIVIC,PIVIC_1, PIBITI, PIVITI])
projetos = projetos[colunas]
projetos = projetos.rename(columns={'Título do Projeto':'TITULO',
                                    'Docente':'DOCENTE',
                                    'Centro':'CENTRO',
                                    'Unidade':'UA',
                                    'Área Conhecimento':'AREA',
                                    'Aluno':'ALUNO',
                                    'CPF Aluno':'CPF_ALUNO',
                                    'Cota Bolsa':'BOLSA'})
projetos.CPF_ALUNO = projetos.CPF_ALUNO.astype(str)
projetos['DOCENTE'] = projetos['DOCENTE'].apply(
    lambda row: unidecode.unidecode(row))

dados_projetos = pd.merge(
    left=projetos, right=servidores,
    left_on='DOCENTE', right_on='Nome').drop('Nome',1)

dados_videos = pd.merge(
    left=dados_projetos, right=formularios,
    left_on='CPF_ALUNO', right_on='CPF_ALUNO')

videos = dados_videos[
    dados_videos['CPF_ORIENTADOR'] == dados_videos['CPF']
]
#Eliminando Duplicatas e ficando com a versão mais recente.
videos_validos = videos.sort_values('TIMESTAMP').drop_duplicates('CPF_ALUNO',keep='last')

#videos_validos['LINK_VIDEO']=videos_validos.apply(lambda row: row['LINK_VIDEO'].split['='][-1])


videos_validos['title'] = videos_validos.apply(lambda row: 
                     'ICT-2020-' + row['AREA'] + '-' + '-' + row['CENTRO'] +
                     '-' + row['UA'] + ':  ' + 
                     str(row['TITULO'])[0 : min(len(str(row['TITULO'])),30)] + '...',
                     axis = 1
                    )

videos_validos['description'] = 'Videos para avaliação dos projetos de Iniciação Científica e Tecnológica, vigência 2019 - 2020, da Universidade Federal de Campina Grande. Provisoriamente armazenados nesta canal por problemas técnicos.'

videos_validos['category'] = '27'

videos_validos['keywords'] = videos_validos.apply(lambda row: 
                     'UFCG,PRPG,PIBIC,PIBITI,' + row['AREA'] +',' + row['CENTRO'],
                     axis = 1
                    )

videos_validos['categorias'] = videos_validos.apply(lambda row: row['CENTRO']+'-'+row['UA'], axis = 1)

In [60]:
videos_validos.head(1)['UA'].iloc[0]

'UAMV'

## Criando Listas

In [61]:
listas = ['CENTRO','UA','AREA']
categorias = [videos_validos[el].unique().tolist() for el in listas]
playlists = [ videos_validos[ videos_validos[el] == categ ]['ID'] 
 for el in listas for categ in categorias[listas.index(el)] ]


In [62]:
column_names = ['TIPO', 'CRITERIO', 'DADOS']
df = pd.DataFrame(columns = column_names)
POS = 0
for item in listas:
    for item2 in categorias[listas.index(item)]:
        df.loc[len(df)] = [item, item2, videos_validos[videos_validos.index.isin(playlists[POS].index)]]
        POS += 1        

## Teste

In [75]:
df1 = df[df['TIPO']=='UA']
df1[df1['CRITERIO']=='UAF'].iloc[0,2]['LINK_VIDEO'].to_list()

['https://drive.google.com/u/0/open?usp=forms_web&id=1MPdpQtkQX7DSimBzgYYpSEm02ngODSQI',
 'https://drive.google.com/u/0/open?usp=forms_web&id=1Ehg__XJTKFx3MP9rnYadSvfo05qz-OtF',
 'https://drive.google.com/u/0/open?usp=forms_web&id=1H9mUF4I0NId4wyEefxXTMhUD3l9yRHdk']

In [66]:
uamv.columns

Index(['ID', 'TITULO', 'DOCENTE', 'CENTRO', 'UA', 'AREA', 'ALUNO', 'CPF_ALUNO',
       'BOLSA', 'TIPO', 'CPF', 'TIMESTAMP', 'EMAIL', 'NOME', 'CPF_ORIENTADOR',
       'LINK_VIDEO', 'title', 'description', 'category', 'keywords'],
      dtype='object')

In [67]:
uamv = uamv[uamv['ID']==7733]

In [75]:
uamv.values[0,-5].split('=')[-1]

'16ubBh2OqK_dVFdRQf0rWw_49WMq2KxZp'

In [25]:
a = 1
a = b

NameError: name 'b' is not defined

In [7]:
np.array(created_playlists).T

array(['UFCG-CSTR-UAMV', 'ERROR'], dtype='<U14')

In [21]:
created_playlists = [['UFCG-CSTR-UAMV','ERROR']]
df = pd.DataFrame(created_playlists, columns=['playlist_title','playlist_id'])

In [22]:
df

,playlist_title,playlist_id
0,UFCG-CSTR-UAMV,ERROR


In [18]:
lista = []
lista.append(created_playlists)


In [28]:
import videolist as vd
from argparse import Namespace
videos_validos = vd.do_list()

In [29]:
videos_validos.head(2)

,ID,TITULO,DOCENTE,CENTRO,UA,AREA,ALUNO,CPF_ALUNO,BOLSA,TIPO,CPF,TIMESTAMP,EMAIL,NOME,CPF_ORIENTADOR,LINK_VIDEO,title,description,category,keywords
46,7733,"CONSUMO, DIGESTIBILIDADE E COMPORTAMENTO INGESTIVO DE CORDEIROS ALIMENTADOS COM DIETAS CONTENDO LÍQUIDO DA CASCA DA CASTANHA DE CAJU (LCC).",LEILSON ROCHA BEZERRA,CSTR,UAMV,ZOOTECNIA,GIOVANA DINIZ LIMA,12169144439,NaN,PIVIC,66045967372,2020-12-21 12:12:12+03:00,giovanadiniz13@gmail.com,Giovana Diniz Lima,66045967372,16ubBh2OqK_dVFdRQf0rWw_49WMq2KxZp,"ICT-2020--PIVIC-ZOOTECNIA--CSTR-UAMV: CONSUMO, DIGESTIBILIDADE E COM...","Videos para avaliação dos projetos de Iniciação Científica e Tecnológica, vigência 2019 - 2020, da Universidade Federal de Campina Grande. Provisoriamente armazenados nesta canal por problemas técnicos.",27,"UFCG,PRPG,PIBIC,PIBITI,ZOOTECNIA,CSTR"
213,7553,VIOLAÇÃO DA INVARIÂNCIA DE LORENTZ: DESENVOLVIMENTOS TEÓRICOS E LIMITES FENOMENOLÓGICOS,EDUARDO MARCOS RODRIGUES DOS PASSOS,CCT,UAF,FÍSICA,CARLOS MIGUEL NÓBREGA DE MEDEIROS,12316973426,NaN,PIVIC,44078927220,2020-12-21 13:56:05+03:00,carlos.miguel@estudante.ufcg.edu.br,Carlos Miguel Nóbrega de Medeiros,44078927220,1MPdpQtkQX7DSimBzgYYpSEm02ngODSQI,ICT-2020--PIVIC-FÍSICA--CCT-UAF: VIOLAÇÃO DA INVARIÂNCIA DE LOR...,"Videos para avaliação dos projetos de Iniciação Científica e Tecnológica, vigência 2019 - 2020, da Universidade Federal de Campina Grande. Provisoriamente armazenados nesta canal por problemas técnicos.",27,"UFCG,PRPG,PIBIC,PIBITI,FÍSICA,CCT"


In [41]:
file = files[0]
file
ID = file.split('/')[-1].split('.')[0]
ID
dados = videos_validos[videos_validos['ID']==int(ID)]
dados

'./VIDEO/7882.mp4'

'7882'

,ID,TITULO,DOCENTE,CENTRO,UA,AREA,ALUNO,CPF_ALUNO,BOLSA,TIPO,CPF,TIMESTAMP,EMAIL,NOME,CPF_ORIENTADOR,LINK_VIDEO,title,description,category,keywords
32,7882,DESENVOLVIMENTO DE MEMBRANAS DE NANOFIBRAS DE TIO2 COM NANOPARTÍCULAS DE AG PARA TRATAMENTO DE ÁGUAS CONTAMINADAS COM CORANTES: PARTE II - UTILIZAÇÃO DA TÉCNICA DE LIOFILIZAÇÃO,ROMUALDO RODRIGUES MENEZES,CCT,UAEMAT,ENGENHARIA DE MATERIAIS E METALÚRGICA,YTALO LUIZ OURIQUES RODRIGUES,10812943457,NaN,PIBITI,02664019457,2020-12-30 20:01:46+03:00,ytaloour@gmail.com,Ytalo Luiz Ouriques Rodrigues,02664019457,1002oE5LjlTc7HM8dRIcxVM2bflxqfb53,ICT-2020--PIBITI-ENGENHARIA DE MATERIAIS E METALÚRGICA--CCT-UAEMAT: DESENVOLVIMENTO DE MEMBRANAS D...,"Videos para avaliação dos projetos de Iniciação Científica e Tecnológica, vigência 2019 - 2020, da Universidade Federal de Campina Grande. Provisoriamente armazenados nesta canal por problemas técnicos.",27,"UFCG,PRPG,PIBIC,PIBITI,ENGENHARIA DE MATERIAIS E METALÚRGICA,CCT"


In [49]:
files = glob('./VIDEO/*.mp4')
keys = []
for file in files:
    ID = file.split('/')[-1].split('.')[0]
    dados = videos_validos[videos_validos['ID']==int(ID)]
    params = Namespace(title=dados['title'].iloc[0],
                        description=dados['description'].iloc[0],
                        category=dados['category'].iloc[0],
                        keywords=dados['keywords'].iloc[0],
                        secret = 'args.secret'
                        )

In [50]:
import youtube
body = youtube.dict_struc(params)

In [51]:
body

{'snippet': {'title': 'ICT-2020--PIBIC-ENGENHARIA DE MATERIAIS E METALÚRGICA--CCT-UAEMAT:  ESTUDO DAS PROPRIEDADES DE COM...',
  'description': 'Videos para avaliação dos projetos de Iniciação Científica e Tecnológica, vigência 2019 - 2020, da Universidade Federal de Campina Grande. Provisoriamente armazenados nesta canal por problemas técnicos.',
  'tags': ['UFCG',
   'PRPG',
   'PIBIC',
   'PIBITI',
   'ENGENHARIA DE MATERIAIS E METALÚRGICA',
   'CCT'],
  'categoryId': '27'},
 'status': {'privacyStatus': 'public'}}

In [52]:
part=",".join(body.keys())
part

'snippet,status'